# Backend



In [135]:
import pandas as pd
import matplotlib.pyplot as plt
from typing import Dict, Tuple

# ==============================================================================
# 0. CÓDIGO EXISTENTE: Modelo de Datos (Cristopher Xander Aza)
# ==============================================================================

def get_month_key(year, month):
    return f'{year}_{month}'

class Account:
    def __init__(self, name:str, typeAccount:str, is_Fixed:bool, sub_cat:str, fixed_value = None):
        self.name = name
        self.typeAccount = typeAccount
        self.sub_category = sub_cat
        self.is_Fixed = is_Fixed
        self.is_closed = False
        self.months = {}
        if is_Fixed:
            if fixed_value != None:
                self.fixed_value = fixed_value
                self.is_closed = True
            else:
                raise ValueError("Valor para cuenta fija no incluido")

    def set_value_on_month(self, year, month, value = None):
        key = get_month_key(year, month)
        if self.is_Fixed:
            self.months[key] = self.fixed_value
        else:
            if value is None:
                raise ValueError("Valor para cuenta en mes no incluido")
            self.months[key] = value

    def get_literal(self, key):
        if self.typeAccount == "egress":
            return 0 - self.months[key]
        else:
            return self.months[key]

class Project:
    def __init__(self, saving_percentage):
        self.incomes = {}
        self.expenses = {}
        self.month_keys = []
        self.saving_percentage = saving_percentage

    def add_income(self, name, sub_category, fixed_value = None):
        is_fixed = fixed_value is not None
        n_income = Account(name, "income", is_fixed, sub_category, fixed_value)
        if name in self.incomes:
            raise ValueError("un ingreso con este nombre ya esta registrado")
        self.incomes[name] = n_income

    def add_egress(self, name, sub_category, fixed_value = None):
        is_fixed = fixed_value is not None
        n_egress = Account(name, "egress", is_fixed, sub_category, fixed_value)
        if name in self.expenses:
            raise ValueError("un egreso con este nombre ya esta registrado")
        self.expenses[name] = n_egress

    def set_value_for_income_in_month(self, month, year,income_name, value = None):
        if income_name not in self.incomes:
            raise ValueError(f'{income_name} no registrado como cuenta de ingreso en este proyecto')
        else:
            key = get_month_key(year, month)
            inc = self.incomes[income_name]
            if key in inc.months:
                raise ValueError("esta cuenta ya posee un valor resgistrado en este mes")
            else:
                inc.set_value_on_month(year, month, value)
            if key not in self.month_keys:
                self.month_keys.append(key)

    def set_value_for_egress_in_month(self, month, year,egress_name, value = None):
        if egress_name not in self.expenses:
            raise ValueError(f'{egress_name} no registrado como cuenta de egreso en este proyecto')
        else:
            key = get_month_key(year, month)
            egress = self.expenses[egress_name]
            if key in egress.months:
                raise ValueError("esta cuenta ya posee un valor resgistrado en este mes")
            else:
                egress.set_value_on_month(year, month, value)
            if key not in self.month_keys:
                self.month_keys.append(key)

    def get_incomes_total_per_month(self, month, year):
        total = 0
        key = get_month_key(year, month)
        if key not in self.month_keys:
            raise ValueError(f'el mes {key} no esta registrado ')
        incomes = self.incomes.values()
        for inc in incomes:
            if inc.is_Fixed:
              total += inc.fixed_value
            elif key not in inc.months:
              raise ValueError(f'el mes no esta cerrado, la cuenta de ingreso {inc.name} no registra valor para este mes')
            else:
              total += inc.months[key]
        return total

    #obtener egresos totales de un mes
    def get_egresses_total_per_month(self, month, year):
      total = 0
      key = get_month_key(year, month)
      if key not in self.month_keys:
        raise ValueError(f'el mes {key} no esta registrado ')
      egresses = self.expenses.values()
      for egress in egresses:
        if egress.is_Fixed:
          total += egress.fixed_value
        elif key not in egress.months:
          raise ValueError(f'el mes no esta cerrado, la cuenta de egreso {egress.name} no registra valor para este mes')
        else:
          total += egress.months[key]
      return total


    def get_total_saving_per_month(self, month, year):
        incomes_total = self.get_incomes_total_per_month(month, year)
        egresses_total = self.get_egresses_total_per_month(month, year)
        return incomes_total - egresses_total

    #obtener los totales ingresados por sub_categoria
    def get_totals_incomes_per_sub_category(self, month, year):
      sub_categories = {}
      key = get_month_key(year, month)
      if key not in self.month_keys:
        raise ValueError(f'el mes {key} no esta registrado ')
      incomes = self.incomes.values()
      for inc in incomes:
        if inc.is_Fixed:
          if inc.sub_category not in sub_categories:
            sub_categories[inc.sub_category] = inc.fixed_value
          else:
            sub_categories[inc.sub_category] += inc.fixed_value
        elif key not in inc.months:
          raise ValueError(f'el mes no esta cerrado, la cuenta de ingreso {inc.name} no registra valor para valor este mes')
        else:
          if inc.sub_category not in sub_categories:
            sub_categories[inc.sub_category] = inc.months[key]
          else:
            sub_categories[inc.sub_category] += inc.months[key]
      return sub_categories

    #obtener los totales egresados por sub_categoria
    def get_totals_egresses_per_sub_category(self, month, year):
      sub_categories = {}
      key = get_month_key(year, month)
      if key not in self.month_keys:
        raise ValueError(f'el mes {key} no esta registrado ')
      egresses = self.expenses.values()
      for egress in egresses:
        if egress.is_Fixed:
          if egress.sub_category not in sub_categories:
            sub_categories[egress.sub_category] = egress.fixed_value
          else:
            sub_categories[egress.sub_category] += egress.fixed_value
        elif key not in egress.months:
          raise ValueError(f'el mes no esta cerrado, la cuenta de egreso {egress.name} no registra valor para este mes')
        else:
          if egress.sub_category not in sub_categories:
            sub_categories[egress.sub_category] = egress.months[key]
          else:
            sub_categories[egress.sub_category] += egress.months[key]
      return sub_categories

    def get_variable_expenses(self):
      expenses = self.expenses.items()
      variable_expenses = [key for key, egress in expenses if not egress.is_Fixed]
      return variable_expenses

    def get_variable_incomes(self):
      incomes = self.incomes.items()
      variable_incomes = [key for key, inc in incomes if not inc.is_Fixed]
      return variable_incomes

# ==============================================================================
# 1. CÓDIGO EXISTENTE: Comparación de Referencia (Wemdry Koralis)
# ==============================================================================

class FinancialBenchmark:
    def __init__(self):
        # Sample benchmark data (can be replaced with real data later)
        self.monthly_income = 4000
        self.monthly_expenses = 3000
        self.saving_rate = (self.monthly_income - self.monthly_expenses) / self.monthly_income

    def compare_to_user(self, user_income, user_expenses):
        user_saving_rate = (user_income - user_expenses) / user_income if user_income > 0 else 0

        print("\n--- Comparación Financiera de Referencia (Genérica) ---")
        print(f"Ingreso Mensual de Referencia: ${self.monthly_income:,.2f}")
        print(f"Gastos Mensuales de Referencia: ${self.monthly_expenses:,.2f}")
        print(f"Tasa de Ahorro de Referencia: {self.saving_rate:.2%}")
        print("-" * 25)
        print(f"Ingreso Mensual del Usuario: ${user_income:,.2f}")
        print(f"Gastos Mensuales del Usuario: ${user_expenses:,.2f}")
        print(f"Tasa de Ahorro del Usuario: {user_saving_rate:.2%}")
        print("-" * 25)

        if user_saving_rate > self.saving_rate:
            print("¡Tus finanzas están por encima del promedio de referencia genérica!")
        elif user_saving_rate < self.saving_rate:
            print("Tus finanzas están por debajo del promedio de referencia genérica. Considera ajustar tus gastos.")
        else:
            print("Tus finanzas están en línea con el promedio de referencia genérica.")

# ==============================================================================
# 2. CÓDIGO NUEVO: Comparación Nacional RD (Integración Solicitada)
# ==============================================================================

class NationalBenchmark:
    """
    Clase para comparar las finanzas del usuario con los promedios nacionales
    de la República Dominicana (RD) en 2025 (datos simulados/estimados).
    """

    # Datos de Referencia Nacional (Hardcodeados y Simulados)
    NATIONAL_AVERAGE_INCOME = 45000.00 # DOP
    NATIONAL_EXPENSE_DISTRIBUTION: Dict[str, float] = {
        "Alimentación": 30.0,
        "Vivienda/Servicios": 25.0,
        "Transporte": 15.0,
        "Salud/Educación": 10.0,
        "Otros Egresos": 20.0     # Ocio, Ropa, etc.
    }
    NATIONAL_SAVING_RATE = 0.15 # 15% de ahorro promedio

    def get_user_monthly_data(self, project_instance: Project, month: str, year: int) -> Tuple[float, float, Dict[str, float]]:
        """ Extrae el ingreso total, el egreso total y la distribución de egresos por subcategoría. """
        try:
            user_total_income = project_instance.get_incomes_total_per_month(month, year)
            user_total_expenses = project_instance.get_egresses_total_per_month(month, year)
            user_expenses_by_sub_cat = project_instance.get_totals_egresses_per_sub_category(month, year)
            return user_total_income, user_total_expenses, user_expenses_by_sub_cat
        except ValueError as e:
            print(f"Error al obtener datos del usuario para el mes {month} {year}: {e}")
            raise

    def compare_and_report(self, project_instance: Project, month: str, year: int):
        """ Calcula métricas, compara con la media nacional y genera el reporte/gráfico. """
        try:
            user_income, user_expenses, user_sub_cat_expenses = self.get_user_monthly_data(project_instance, month, year)
        except ValueError:
            return

        # 1. CÁLCULO DE MÉTRICAS DEL USUARIO
        user_net_income = user_income - user_expenses
        user_saving_rate = user_net_income / user_income if user_income > 0 else 0

        # Filtra 'Ahorro' e 'Inversión' para calcular solo 'gastos de vida' para el pie chart
        user_life_expenses = {
            k: v for k, v in user_sub_cat_expenses.items()
            if k not in ["Ahorro", "Inversión"] # Excluye ahorro/inversión de los gastos de vida
        }
        user_total_life_expenses = sum(user_life_expenses.values())

        # Creación de la distribución de gastos para el gráfico (porcentaje sobre el GASTO DE VIDA)
        user_expense_distribution: Dict[str, float] = {}
        if user_total_life_expenses > 0:
            for cat in self.NATIONAL_EXPENSE_DISTRIBUTION:
                # Asume que las subcategorías del usuario están mapeadas a las nacionales
                val = user_life_expenses.get(cat, 0.0)
                user_expense_distribution[cat] = (val / user_total_life_expenses) * 100
        else:
            # Si no hay gastos de vida registrados (solo Ahorro/Inversión), pone 0% en todo
            for cat in self.NATIONAL_EXPENSE_DISTRIBUTION:
                 user_expense_distribution[cat] = 0.0

        # 2. REPORTE Y COMPARACIÓN EN CONSOLA
        print("\n" + "="*50)
        print(f"       REPORTE DE COMPARACIÓN CON MEDIA NACIONAL RD ({month.upper()}, {year})")
        print("="*50)

        # Comparación de Ingreso
        income_diff = user_income - self.NATIONAL_AVERAGE_INCOME
        income_percent_diff = (income_diff / self.NATIONAL_AVERAGE_INCOME) * 100

        income_status = "superior a" if income_diff > 0 else "inferior a"
        print(f"💰 Ingreso Mensual del Usuario (DOP): {user_income:,.2f}")
        print(f"   Media Nacional (DOP): {self.NATIONAL_AVERAGE_INCOME:,.2f}")
        print(f"   Tu ingreso es un {abs(income_percent_diff):.2f}% {income_status} la media nacional.")

        # Comparación de Tasa de Ahorro
        saving_diff = user_saving_rate - self.NATIONAL_SAVING_RATE
        saving_status = "superior a" if saving_diff > 0 else "inferior a"
        print("-" * 50)
        print(f"📈 Tasa de Ahorro Neto (Ahorro Total / Ingreso Total):")
        print(f"   Tu Tasa de Ahorro: {user_saving_rate:.2%}")
        print(f"   Tasa Nacional Promedio: {self.NATIONAL_SAVING_RATE:.2%}")
        print(f"   Tu tasa es {abs(saving_diff):.2%} {saving_status} la media nacional.")
        print("="*50)

        # 3. Gráfico de Distribución de Gastos
        self._plot_expense_distribution(user_expense_distribution)


    def _plot_expense_distribution(self, user_distribution: Dict[str, float]):
        """ Genera un gráfico de barras comparando la distribución de gastos. """

        national_data = self.NATIONAL_EXPENSE_DISTRIBUTION
        all_categories = list(national_data.keys())

        # Crear el DataFrame para Matplotlib
        df_data = {
            'Categoría': all_categories,
            'Usuario (%)': [user_distribution.get(cat, 0.0) for cat in all_categories],
            'Nacional (%)': [national_data[cat] for cat in all_categories]
        }

        df = pd.DataFrame(df_data).set_index('Categoría')

        # Generar el gráfico
        plt.figure(figsize=(10, 6))
        df.plot(kind='bar', figsize=(12, 7), alpha=0.8)

        plt.title('Distribución de Gastos de Vida: Usuario vs. Media Nacional (RD)', fontsize=16)
        plt.ylabel('Porcentaje del Gasto Total de Vida (%)', fontsize=12)
        plt.xlabel('Categoría de Gasto', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.legend(title='Referencia', fontsize=10)
        plt.grid(axis='y', linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()

# ==============================================================================
# 3. CÓDIGO CORREGIDO: (wendry) Ejecución y Registro de Datos (Triana Olividia Garcia J. - VERSIÓN PROYECCIÓN BASE MES 1)
# ==============================================================================

# Inicializar las variables para evitar NameError en caso de fallo crítico
porcentaje_total = 0.0
porcentaje_ahorro = 0.0
porcentaje_inversion = 0.0
ingreso_mensual = 0.0

# --- Entrada de datos ---
print("\n--- Configuración Inicial ---")
try:
    ingreso_raw = input("Ingresa tus ingresos netos mensuales (DOP): ").replace(',', '').strip()
    ingreso_mensual = float(ingreso_raw)

    porc_total_raw = input("¿Qué porcentaje de sus ingresos desea ahorrar/invertir mensualmente?: ").replace(',', '').strip()
    porc_total = float(porc_total_raw)

    # Validamos el porcentaje total
    if porc_total < 0 or porc_total > 100:
        raise ValueError("El porcentaje total debe estar entre 0 y 100")

    porcentaje_total = porc_total / 100.0

    porc_ahorro_raw = input("¿Qué porcentaje de ese ahorro total desea destinar al ahorro bruto?: ").replace(',', '').strip()
    porc_ahorro = float(porc_ahorro_raw)

    # Validamos el porcentaje de ahorro (dentro del total)
    if porc_ahorro < 0 or porc_ahorro > 100:
        raise ValueError("El porcentaje de ahorro (dentro del total) debe estar entre 0 y 100")

    porcentaje_ahorro = porc_ahorro / 100.0
    porcentaje_inversion = 1.0 - porcentaje_ahorro

except ValueError as e:
    print(f"Error en la entrada de datos: {e}")
    exit()

# --- Calcular montos ---
ahorro_mensual = ingreso_mensual * porcentaje_total * porcentaje_ahorro
inversion_mensual = ingreso_mensual * porcentaje_total * porcentaje_inversion

print(f"\nCon un ingreso mensual de {ingreso_mensual:,.2f} y un ahorro total del {porc_total:.2f}%,")
print(f"usted debe ahorrar {ahorro_mensual:,.2f} DOP mensualmente para ahorro bruto y {inversion_mensual:,.2f} DOP para inversión.")

# --- Crear proyecto y cuentas de Ahorro/Inversión ---
proyecto = Project(saving_percentage=porcentaje_total)
proyecto.add_income("Ingreso mensual", "Sueldo", ingreso_mensual)
proyecto.add_egress("Ahorro bruto", "Ahorro", ahorro_mensual)
proyecto.add_egress("Inversión mensual", "Inversión", inversion_mensual)

# ==============================================================================
# NUEVO: REGISTRO DE EGRESOS DE VIDA (PARA COMPARACIÓN NACIONAL)
# Las subcategorías deben coincidir con las del NationalBenchmark:
# "Alimentación", "Vivienda/Servicios", "Transporte", "Salud/Educación", "Otros Egresos"
# ==============================================================================

# 1. Definir EGRESOS FIJOS de Vida y agregarlos al proyecto
proyecto.add_egress("Alquiler", "Vivienda/Servicios", 10000.00)
proyecto.add_egress("Luz/Agua/Tel", "Vivienda/Servicios", 3000.00)

# 2. Definir EGRESOS VARIABLES (Necesitan un valor de entrada en Enero) y agregarlos
proyecto.add_egress("Mercado", "Alimentación")
proyecto.add_egress("Gasolina", "Transporte")
proyecto.add_egress("Farmacia", "Salud/Educación")
proyecto.add_egress("Ocio", "Otros Egresos")

# Almacenar los egresos variables de Enero
egresos_variables_enero = {}
print("\n--- Ingreso de Egresos de Vida Base (Mes de Enero) ---")
egresos_vars = ["Mercado", "Gasolina", "Farmacia", "Ocio"]

for nombre_egreso in egresos_vars:
    raw_value = input(f"Gasto en {nombre_egreso} para Enero 2025 (DOP): ").replace(',', '').strip()
    try:
        valor = float(raw_value)
        egresos_variables_enero[nombre_egreso] = valor
    except ValueError:
        print(f"Valor no válido ingresado para {nombre_egreso}. Se usará 0.00.")
        egresos_variables_enero[nombre_egreso] = 0.00


# --- Registrar valores por mes (PROYECCIÓN A 12 MESES) ---
meses = ["Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre"]

for i, mes in enumerate(meses, start=1):
    # Registro de Ingresos/Ahorro Fijo
    proyecto.set_value_for_income_in_month(mes, 2025, "Ingreso mensual", ingreso_mensual)
    proyecto.set_value_for_egress_in_month(mes, 2025, "Ahorro bruto", ahorro_mensual)
    proyecto.set_value_for_egress_in_month(mes, 2025, "Inversión mensual", inversion_mensual)

    # Registro de Egresos Fijos de Vida (Se aplica a todos los meses)
    proyecto.set_value_for_egress_in_month(mes, 2025, "Alquiler", 10000.00)
    proyecto.set_value_for_egress_in_month(mes, 2025, "Luz/Agua/Tel", 3000.00)

    # Registro de Egresos VARIABLES (Proyección: se usa el valor base de Enero para todos los meses)
    for nombre_egreso, valor_enero in egresos_variables_enero.items():
        proyecto.set_value_for_egress_in_month(mes, 2025, nombre_egreso, valor_enero)

print("\nProyección de 12 meses completada usando los gastos variables de Enero como base.")

# --- Crear DataFrame de ahorro acumulado ---
# Now that variable expenses are set for each month, we can calculate for all months
ahorro_acumulado = [proyecto.get_total_saving_per_month(mes, 2025) for i, mes in enumerate(meses)]
df = pd.DataFrame({"Mes": meses, "Ahorro neto mensual (DOP)": ahorro_acumulado})


# --- Gráficas de Ahorro/Proyección --- (Parte existente)
# Nota: Aquí falta la lógica para que el usuario ingrese la META, la he omitido
# por simplicidad en el código final, pero se mantiene la estructura.
meta = 120000.0 # Valor Hardcodeado para evitar otro input

# 1) Avance actual (mes 1) vs Meta
ahorro_mensual_neto = proyecto.get_total_saving_per_month("Enero", 2025) # Ahorro real del mes 1
plt.figure(figsize=(7,5))
plt.bar(["Ahorro Neto (1 mes)", "Meta"], [ahorro_mensual_neto, meta])
plt.title("Avance actual vs Meta")
plt.ylabel("Pesos dominicanos (DOP)")
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.show()

# 2) Proyección 12 meses vs Meta (Usa la columna 'Ahorro neto mensual' del DataFrame)
# Necesitas calcular la proyección acumulada:
ahorro_acumulado_proj = [sum(ahorro_acumulado[:i+1]) for i in range(len(ahorro_acumulado))]
df_proj = pd.DataFrame({"Mes": meses, "Ahorro acumulado (DOP)": ahorro_acumulado_proj})

plt.figure(figsize=(10,6))
plt.plot(df_proj["Mes"], df_proj["Ahorro acumulado (DOP)"], marker='o', label="Ahorro acumulado")
plt.axhline(y=meta, color='red', linestyle='--', label="Meta")
plt.title("Proyección de ahorro a 12 meses")
plt.xlabel("Mes")
plt.ylabel("Ahorro acumulado (DOP)")
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()

# ==============================================================================
# 4. EJECUCIÓN DE LAS COMPARATIVAS
# ==============================================================================

# 4.1 Uso de la Comparación Genérica (Código Existente de Wemdry)
benchmark = FinancialBenchmark()
mes_comp = "Enero"
ano_comp = 2025
try:
    user_total_income = proyecto.get_incomes_total_per_month(mes_comp, ano_comp)
    user_total_expenses = proyecto.get_egresses_total_per_month(mes_comp, ano_comp)
    benchmark.compare_to_user(user_total_income, user_total_expenses)
except ValueError as e:
    print(f"Error al obtener datos para la comparación genérica: {e}")


# 4.2 Uso de la Comparación Nacional RD (Código Nuevo)
benchmark_nacional = NationalBenchmark()
try:
    # We can now compare for any month after the loop has run
    benchmark_nacional.compare_and_report(proyecto, "Febrero", 2025) # Example for February
except Exception as e:
    print(f"No se pudo generar el reporte nacional: {e}") #primera segunda o tercera? si


--- Configuración Inicial ---


KeyboardInterrupt: Interrupted by user

# Form Reutilizable para agregar ingresos/egresos

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output


def cargar_form(b, nombre:str, tipo_accion:str):
  clear_output(wait=True)
  # Esto carga la librería de íconos Font Awesome
  display(HTML("""
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
  """))

  valor_fijo = tipo_accion == "ingreso_fijo" or tipo_accion == "egreso_fijo"

  # 1. Crear los widgets de input

  # Label con el nombre de la accion

  name_label = widgets.Label(value= nombre)

  # Input de Nombre (Texto)
  nombre_input = widgets.Text(
      description=f'Nombre:',
      placeholder='Escribe aquí',
      layout=widgets.Layout(width='250px') # Ajuste de ancho
  )

  #Select de Subcategorias (Dropdown)
  subcategoria_select = widgets.Dropdown(
      options= subcategorias,
      value='Alimentación',
      description='Subcategoría:',
      layout=widgets.Layout(width='500px') # Ajuste de ancho
  )

  # 2. Crear el botón de enviar
  enviar_boton = widgets.Button(
      description='Enviar',
      button_style='success', # Opcional: para darle un color verde
      icon='fa-check',
      layout=widgets.Layout(width='250px') # Ajuste de ancho
  )

  # 3. Crear el botón de volver al menu
  back_btn = widgets.Button(
      description='Volver atrás',
      button_style='primary',
      icon='fa-arrow-left',
      layout=widgets.Layout(width='250px') # Ajuste de ancho
  )

  # Asignar la función al evento de click del botón
  #enviar_boton.on_click(on_enviar_clicked)
  back_btn.on_click(show_config_ui)

  # 3. Organizar la interfaz

  # Agrupar los dos inputs uno al lado del otro usando HBox (Horizontal Box)
  buttons_row = widgets.HBox([back_btn, enviar_boton])

  if (valor_fijo):
    # Input de Valor (Número entero)
    valor_input = widgets.BoundedIntText(
        description='Valor:',
        min=0,
        max=100000000000,
        step=1,
        value=0,
        layout=widgets.Layout(width='250px') # Ajuste de ancho
    )
    input_row = widgets.HBox([nombre_input, valor_input])

    # Agrupar la fila de inputs, el botón y el área de salida usando VBox (Vertical Box)
    # VBox los apila verticalmente
    form_layout = widgets.VBox([
        name_label,
        input_row,
        subcategoria_select,
        buttons_row,
    ], layout=widgets.Layout(
        grid_gap= '10px'
    ))

    match tipo_accion:
      case "ingreso_fijo":
        enviar_boton.on_click(lambda b: form_navegacion(tipo_accion, nombre_input.value, subcategoria_select.value, valor_input.value))

      case "egreso_fijo":
        enviar_boton.on_click(lambda b: form_navegacion(tipo_accion, nombre_input.value, subcategoria_select.value, valor_input.value))
  else:
    input_row = widgets.HBox([nombre_input, subcategoria_select])
    # Agrupar la fila de inputs, el botón y el área de salida usando VBox (Vertical Box)
    # VBox los apila verticalmente
    form_layout = widgets.VBox([
        name_label,
        input_row,
        buttons_row,
    ], layout=widgets.Layout(
        grid_gap= '10px'
    ))

    match tipo_accion:
      case "ingreso_variable":
        enviar_boton.on_click(lambda b: form_navegacion(tipo_accion, nombre_input.value, subcategoria_select.value))
      case "egreso_variable":
        enviar_boton.on_click(lambda b: form_navegacion(tipo_accion, nombre_input.value, subcategoria_select.value))

  # Mostrar la interfaz
  display(form_layout)

In [ ]:
def form_navegacion(accion, nombre, subcategoria, valor = None):
  match accion:
    case "ingreso_variable":
      proyecto.add_income(nombre, subcategoria)
      show_config_ui()
    case "egreso_variable":
      proyecto.add_egress(nombre, subcategoria)
      show_config_ui()
    case "ingreso_fijo":
      proyecto.add_income(nombre, subcategoria, valor)
      show_config_ui()
    case "egreso_fijo":
      proyecto.add_egress(nombre, subcategoria, valor)
      show_config_ui()

# Porcentaje de Ahorro UI

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output


def cargar_porcentaje_ahorro_form(b):
  clear_output(wait=True)
  # Esto carga la librería de íconos Font Awesome
  display(HTML("""
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
  """))

  # 1. Crear los widgets de input

  # Label con el nombre de la accion

  name_label = widgets.Label(value= "Porcentaje de Ahorro")

  # Input de Valor (Número entero)
  valor_input = widgets.BoundedIntText(
      description='Valor:',
      min=0,
      max=100, # Puedes ajustar el rango máximo
      step=1,
      value=0,
      layout=widgets.Layout(width='250px') # Ajuste de ancho
  )

  if proyecto.saving_percentage is not None:
    valor_input.value = proyecto.saving_percentage

  # 2. Crear el botón de enviar
  enviar_boton = widgets.Button(
      description='Enviar',
      button_style='success', # Opcional: para darle un color verde
      icon='fa-check',
      layout=widgets.Layout(width='125px') # Ajuste de ancho
  )

  # 3. Crear el botón de volver al menu
  back_btn = widgets.Button(
      description='Volver atrás',
      button_style='primary',
      icon='fa-arrow-left',
      layout=widgets.Layout(width='125px') # Ajuste de ancho
  )

  # Asignar la función al evento de click del botón
  enviar_boton.on_click(lambda b: set_saving_porcentage(valor_input.value))
  back_btn.on_click(show_config_ui)

  # 3. Organizar la interfaz

  # Agrupar los dos inputs uno al lado del otro usando HBox (Horizontal Box)
  buttons_row = widgets.HBox([back_btn, enviar_boton])

  # Agrupar la fila de inputs, el botón y el área de salida usando VBox (Vertical Box)
  # VBox los apila verticalmente
  form_layout = widgets.VBox([
      name_label,
      valor_input,
      buttons_row,
  ], layout=widgets.Layout(
      grid_gap= '10px'
  ))

  # Mostrar la interfaz
  display(form_layout)

  if proyecto.saving_percentage is None:
    print("Debes agregar un porcentaje de ahorro para registrar ingresos y egresos")

# Config Router

In [ ]:
def navegar(b, accion):
  if proyecto.saving_percentage is None:
    cargar_porcentaje_ahorro_form(b)
    return

  match accion:
    case "ingreso_fijo":
      cargar_form(b, "Ingreso Fijo", "ingreso_fijo")
      return
    case "ingreso_variable":
      cargar_form(b, "Ingreso Variable", "ingreso_variable")
      return
    case "egreso_fijo":
      cargar_form(b, "Egreso Fijo", "egreso_fijo")
      return
    case "egreso_variable":
      cargar_form(b, "Egreso Variable", "egreso_variable")
    case "porcentaje_ahorro":
      cargar_porcentaje_ahorro_form(b)


# Config UI

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output


def show_config_ui(b= None):
  clear_output(wait=True)

  display(HTML("""
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
  """))

  # Creacion de botones
  add_ingreso_fijo_btn = widgets.Button(description='Añadir Ingreso Fijo', button_style='info', icon='fa-plus', layout=widgets.Layout(width='200px', height='50px'))
  add_ingreso_var_btn = widgets.Button(description='Añadir Ingreso Variable', button_style='success', icon='fa-plus', layout=widgets.Layout(width='200px', height='50px'))
  add_egreso_fijo_btn = widgets.Button(description='Añadir Egreso Fijo', button_style='danger', icon='fa-plus', layout=widgets.Layout(width='200px', height='50px'))
  add_egreso_var_btn = widgets.Button(description='Añadir Egreso Variable', button_style='warning', icon='fa-plus', layout=widgets.Layout(width='200px', height='50px'))
  config_porcentaje_ahorro_btn = widgets.Button(description='Configurar Porcentaje de Ahorro', button_style='primary', icon='fa-gear', layout=widgets.Layout(width='auto', height='60px'))
  back_to_menu = widgets.Button(description='Volver al Menú Principal', button_style='primary', icon='fa-arrow-left', layout=widgets.Layout(width='auto', height='60px'))

  # Asignacion de acciones
  back_to_menu.on_click(main_menu)
  add_ingreso_fijo_btn.on_click(lambda b: navegar(b, "ingreso_fijo"))
  add_ingreso_var_btn.on_click(lambda b: navegar(b, "ingreso_variable"))
  add_egreso_fijo_btn.on_click(lambda b: navegar(b, "egreso_fijo"))
  add_egreso_var_btn.on_click(lambda b: navegar(b, "egreso_variable"))
  config_porcentaje_ahorro_btn.on_click(lambda b: navegar(b, "porcentaje_ahorro"))

  # --- Layout de configuraciones

  right_panel = widgets.VBox([
      back_to_menu,
      widgets.HBox([
          add_ingreso_fijo_btn,
          add_ingreso_var_btn
      ]),
      widgets.HBox([
          add_egreso_fijo_btn,
          add_egreso_var_btn
      ]),
        config_porcentaje_ahorro_btn
  ])

  # --- Combined Layout ---
  main_layout = widgets.HBox([
      right_panel
  ])

  display(main_layout)

# Form reutilizable para registrar ingresos y egresos

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output


def cargar_form_registro(b, tipo):
  clear_output(wait=True)

  display(HTML("""
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
  """))

  # Label con el nombre de la accion

  name_label = widgets.Label(value= tipo)

  if tipo == "Ingresos":
    cuentas = proyecto.get_variable_incomes()
  else:
    cuentas = proyecto.get_variable_expenses()

  # Input de Nombre (Texto)
  incomes_select = widgets.Dropdown(
      options= cuentas,
      description='Cuenta:',
      layout=widgets.Layout(width='250px') # Ajuste de ancho
  )

  #Select de Subcategorias (Dropdown)
  meses_select = widgets.Dropdown(
      options= meses_proyecto,
      value='Enero',
      description='Mes:',
      layout=widgets.Layout(width='250px') # Ajuste de ancho
  )

  valor_input = widgets.BoundedIntText(
      description='Valor:',
      min=0,
      max=100000000000,
      step=1,
      value=0,
      layout=widgets.Layout(width='250px') # Ajuste de ancho
  )

  year_input = widgets.BoundedIntText(
      description='Año:',
      min=0,
      max=5000,
      step=1,
      value=0,
      layout=widgets.Layout(width='250px') # Ajuste de ancho
  )

  selects_row = widgets.HBox([incomes_select, meses_select])
  inputs_row = widgets.HBox([valor_input, year_input])

  # Creacion de botones
  back_to_menu = widgets.Button(description='Volver al Menú Principal', button_style='primary', icon='fa-arrow-left', layout=widgets.Layout(width='250px'))

  # 2. Crear el botón de enviar
  enviar_boton = widgets.Button(
      description='Enviar',
      button_style='success', # Opcional: para darle un color verde
      icon='fa-check',
      layout=widgets.Layout(width='250px') # Ajuste de ancho
  )

  buttons_row = widgets.HBox([back_to_menu, enviar_boton])

  # Asignacion de acciones
  back_to_menu.on_click(main_menu)

  if tipo == "Ingresos":
    enviar_boton.on_click(lambda b: form_registro_nav(tipo, meses_select.value, year_input.value, incomes_select.value, valor_input.value))
  else:
    enviar_boton.on_click(lambda b: form_registro_nav(tipo, meses_select.value, year_input.value, incomes_select.value, valor_input.value))

  # --- Layout de configuraciones

  right_panel = widgets.VBox([
      name_label,
      selects_row,
      inputs_row,
      buttons_row,
  ], layout=widgets.Layout(
          grid_gap= '10px'
      ))

  # --- Combined Layout ---
  main_layout = widgets.HBox([
      right_panel
  ])

  display(main_layout)

  # for key, income in proyecto.incomes.items():
  #   print(income.months)

  # for key, expense in proyecto.expenses.items():
  #   print(expense.months)

In [ ]:
def form_registro_nav(tipo, mes, ano, name, value):
  match tipo:
    case "Ingresos":
      proyecto.set_value_for_income_in_month(mes, ano, name, value)
      main_menu()
    case "Egresos":
      proyecto.set_value_for_egress_in_month(mes, ano, name, value)
      main_menu()

# Menu Principal


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output


def main_menu(b = None):
  # --- Right Panel (Dashboard and Navigation) ---
  clear_output(wait=True)
  dashboard_button = widgets.Button(description='Dashboard', button_style='info', layout=widgets.Layout(width='auto', height='50px'))
  register_income_button = widgets.Button(description='Registrar Ingresos', button_style='success', layout=widgets.Layout(width='auto', height='50px'))
  register_expense_button = widgets.Button(description='Registrar Egreso', button_style='danger', layout=widgets.Layout(width='auto', height='50px'))
  config_button = widgets.Button(description='Config (Ingresos/Egresos Fijos, Categorías y metas de ahorro)', button_style='warning', layout=widgets.Layout(width='auto', height='50px'))

  analysis_metas_button = widgets.Button(description='Análisis contra Metas de Ahorro', button_style='primary', layout=widgets.Layout(width='auto', height='60px'))
  analysis_poblacion_button = widgets.Button(description='Análisis contra datos de la población', button_style='primary', layout=widgets.Layout(width='auto', height='60px'))

  right_panel = widgets.VBox([
      dashboard_button,
      register_income_button,
      register_expense_button,
      config_button,
      widgets.HBox([
          analysis_metas_button,
          analysis_poblacion_button
      ])
  ])

  config_button.on_click(show_config_ui)
  register_income_button.on_click(lambda b: cargar_form_registro(b, "Ingresos"))
  register_expense_button.on_click(lambda b: cargar_form_registro(b, "Egresos"))
  analysis_metas_button.on_click(lambda b : generar_grafica_1())
  analysis_poblacion_button.on_click(lambda b : generar_grafica_comparativa())

  # --- Combined Layout ---
  main_layout = widgets.HBox([
      right_panel
  ])

  display(main_layout)

# Variables Bases del Proyecto

In [ ]:
proyecto = Project(None);

def set_saving_porcentage(saving_percentage):
    proyecto.saving_percentage = saving_percentage
    show_config_ui()


subcategorias = ["Alimentación", "Vivienda/Servicios", "Transporte", "Salud/Educación", "Otros"]
meses_proyecto = ["Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre"]

# Generación de Gráficas

In [ ]:
import ipywidgets as widgets
from ipywidgets import Output
from IPython.display import display, HTML, clear_output


# ==============================================================================
# 3. CÓDIGO CORREGIDO: (wendry) Ejecución y Registro de Datos (Triana Olividia Garcia J. - VERSIÓN PROYECCIÓN BASE MES 1)
# ==============================================================================

def generar_grafica_1():
  clear_output(wait=True)
  porcentaje_total = 0.0
  porcentaje_ahorro = 0.0
  porcentaje_inversion = 0.0
  ingreso_mensual = proyecto.get_incomes_total_per_month("Enero", 2025)

  porcentaje_ahorro = proyecto.saving_percentage / 100.0
  porcentaje_total = porcentaje_ahorro
  porcentaje_inversion = 1.0 - porcentaje_ahorro

  # except ValueError as e:
  #     print(f"Error en la entrada de datos: {e}")
  #     exit()

  # --- Calcular montos ---
  ahorro_mensual = ingreso_mensual * porcentaje_total * porcentaje_ahorro
  inversion_mensual = ingreso_mensual * porcentaje_total * porcentaje_inversion

  print(f"\nCon un ingreso mensual de {ingreso_mensual:,.2f} y un ahorro total del {porc_total:.2f}%,")
  print(f"usted debe ahorrar {ahorro_mensual:,.2f} DOP mensualmente para ahorro bruto y {inversion_mensual:,.2f} DOP para inversión.")

  # --- Registrar valores por mes (PROYECCIÓN A 12 MESES) ---
  meses = ["Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre"]

  # --- Crear DataFrame de ahorro acumulado ---
  # Now that variable expenses are set for each month, we can calculate for all months
  ahorro_acumulado = [proyecto.get_total_saving_per_month("Enero", 2025) for i, mes in enumerate(meses_proyecto)]
  df = pd.DataFrame({"Mes": meses, "Ahorro neto mensual (DOP)": ahorro_acumulado})


  # --- Gráficas de Ahorro/Proyección --- (Parte existente)
  # Nota: Aquí falta la lógica para que el usuario ingrese la META, la he omitido
  # por simplicidad en el código final, pero se mantiene la estructura.
  meta = 120000.0 # Valor Hardcodeado para evitar otro input

  # 1) Avance actual (mes 1) vs Meta
  out_grafica_1 = Output()
  with out_grafica_1:
    ahorro_mensual_neto = proyecto.get_total_saving_per_month("Enero", 2025) # Ahorro real del mes 1
    plt.figure(figsize=(7,5))
    plt.bar(["Ahorro Neto (1 mes)", "Meta"], [ahorro_mensual_neto, meta])
    plt.title("Avance actual vs Meta")
    plt.ylabel("Pesos dominicanos (DOP)")
    plt.grid(axis='y', linestyle='--', alpha=0.3)
    plt.show()

  back_to_menu = widgets.Button(description='Volver al Menú Principal', button_style='primary', icon='fa-arrow-left', layout=widgets.Layout(width='250px', height='50px'))
  back_to_menu.on_click(main_menu)

  # 2) Proyección 12 meses vs Meta (Usa la columna 'Ahorro neto mensual' del DataFrame)
  # Necesitas calcular la proyección acumulada:
  ahorro_acumulado_proj = [sum(ahorro_acumulado[:i+1]) for i in range(len(ahorro_acumulado))]
  df_proj = pd.DataFrame({"Mes": meses, "Ahorro acumulado (DOP)": ahorro_acumulado_proj})

  out_grafica_2 = Output()
  with out_grafica_2:
    plt.figure(figsize=(10,6))
    plt.plot(df_proj["Mes"], df_proj["Ahorro acumulado (DOP)"], marker='o', label="Ahorro acumulado")
    plt.axhline(y=meta, color='red', linestyle='--', label="Meta")
    plt.title("Proyección de ahorro a 12 meses")
    plt.xlabel("Mes")
    plt.ylabel("Ahorro acumulado (DOP)")
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    plt.show()

  graficas_row = widgets.HBox([
      out_grafica_1,
      out_grafica_2
  ],
      layout=widgets.Layout(
          grid_gap= '10px')
  )
  main_layout = widgets.VBox([
      graficas_row,
      back_to_menu
  ],
      grid_gap= '10px'
  )
  display(main_layout)

def generar_grafica_comparativa():
  clear_output(wait=True)
  # ==============================================================================
  # 4. EJECUCIÓN DE LAS COMPARATIVAS
  # ==============================================================================

  # 4.1 Uso de la Comparación Genérica (Código Existente de Wemdry)
  benchmark = FinancialBenchmark()
  mes_comp = "Enero"
  ano_comp = 2025
  try:
      user_total_income = proyecto.get_incomes_total_per_month(mes_comp, ano_comp)
      user_total_expenses = proyecto.get_egresses_total_per_month(mes_comp, ano_comp)
      benchmark.compare_to_user(user_total_income, user_total_expenses)
  except ValueError as e:
      print(f"Error al obtener datos para la comparación genérica: {e}")


  # 4.2 Uso de la Comparación Nacional RD (Código Nuevo)
  benchmark_nacional = NationalBenchmark()
  try:
      # We can now compare for any month after the loop has run
      benchmark_nacional.compare_and_report(proyecto, "Enero", 2025) # Example for February
  except Exception as e:
      print(f"No se pudo generar el reporte nacional: {e}") #primera segunda o tercera? si

  back_to_menu = widgets.Button(description='Volver al Menú Principal', button_style='primary', icon='fa-arrow-left', layout=widgets.Layout(width='250px', height='50px'))
  back_to_menu.on_click(main_menu)
  main_layout = widgets.HBox([
      back_to_menu
  ],
      grid_gap= '10px'
  )

  display(main_layout)

#Ejecución del Proyecto

In [152]:
# Inicio del programa

main_menu()